In [44]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### CSV to DataFrame

In [2]:
df = pd.read_csv("/Users/suchan/study/파이널 프로젝트/contents_Watcha.csv")

In [3]:
df['title'].unique()

array(['우리들의 블루스', '파친코', '돼지의 왕', '그린마더스클럽', '사내 맞선', '스물다섯 스물하나',
       '시맨틱 에러', '소년비행', '악의 마음을 읽는 자들', '나의 아저씨', '소년심판', '비밀의 숲',
       '비밀의 숲 2', '멜로가 체질', '괴물', '모범택시', '미스터 션샤인', '트레이서 시즌 1',
       '트레이서 시즌 2', '이렇게 된 이상 청와대로 간다', '술꾼도시여자들', '라이프 온 마스', '블랙독',
       '해피니스', '태종 이방원', '도시남녀의 사랑법', '검색어를 입력하세요 WWW', '옷소매 붉은 끝동',
       '악의 꽃', '런 온', '손 the guest', '서른, 아홉', '슬기로운 의사생활',
       '슬기로운 의사생활 시즌 2', 'D.P.', '사랑의 불시착', '스토브리그', '왓쳐', '그림자 미녀',
       '동백꽃 필 무렵', '백일의 낭군님', '어느 날 우리 집 현관으로 멸망이 들어왔다', '카이로스',
       '보좌관 – 세상을 움직이는 사람들', '보좌관 2 - 세상을 움직이는 사람들', '기름진 멜로', '새빛남고 학생회',
       '좋좋소', '키마이라', '며느라기 시즌 1', '며느라기 2…ing', '김비서가 왜 그럴까', '부부의 세계',
       '그녀의 사생활', '로스쿨', '구경이', '무브 투 헤븐: 나는 유품정리사입니다', '마우스', '검법남녀',
       '검법남녀 시즌2', '엉클', '하늘에서 내리는 1억개의 별', '우수무당 가두심', '오징어 게임',
       '킹덤 시즌 1', '킹덤 시즌 2', 'SKY 캐슬', '호텔 델루나', '갯마을 차차차', '아는 와이프',
       '원 더 우먼', '보이스', '보이스2', '보이스3', '오월의 청춘', '이 구역의 미친 X', '라켓소년단',
       '언프레임드', '마녀식당으로 오세요', '연애미수',

In [4]:
df[df['title']==df['title'].unique()[0]]['reviews']

0      이 화려한 캐스팅 라인업을 보고 안 볼수가 있나.. ㄷㄷㄷ 게다가 노희경 극본..ㄷ...
1                 침묵의 순간들이 더욱 처연하게 빛나는, 노희경 극본의 또 다른 경지.
2              세상에..노희경 작가와 이 배우들이라면 어떤 스토리가 됐든 꼭 보고싶다..
3      하 진짜 좋아하는 배우 전부 다 나오네…\n.\n한수의 위험한 생각이 뭔지 알 것 ...
4      사람 사는 거 특별할 거 없다지만, 저마다의 사정을 찬찬히 듣노라면 그것만큼 재미난...
                             ...                        
102                                                 존나기대
103                                      노희경 작가님✖️💙🤍💙🤍💙🤍
104                                                   존버
105    인생살이 적재적소한 절대적인 필요악\n엿같은 인간세상 내가 살기위해\n인간사회에 도...
106                                            영화가 따로 없네
Name: reviews, Length: 107, dtype: object

### 전처리 함수

In [5]:
import re
from konlpy.tag import Mecab
from tqdm import tqdm

In [6]:
def clean_text(text):
    text = text.replace(".", "").strip()
    text = text.replace("·", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9]+'
    text = re.sub(pattern, '', text)
    return text

In [11]:
def get_nouns(tokenizer, sentence):
    tagged = tokenizer.pos(sentence)
    nouns = [word for word, tag in tagged if tag in ['NNG', 'NNP'] and len(word) >1]
    return nouns

def tokenize(df):
    tokenizer = Mecab()
    processed_data = []
    for sent in tqdm(df['reviews']):
        sentence = clean_text(str(sent).replace("\n", "").strip())
        processed_data.append(get_nouns(tokenizer, sentence))
    return processed_data

### 전처리 실행

In [13]:
preprocessed_data = tokenize(df)

100%|██████████| 91249/91249 [00:08<00:00, 10270.90it/s]


In [17]:
df['preprocessed_reviews'] = preprocessed_data

### LDA

In [62]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel

In [64]:
title = '한 번 다녀왔습니다'

In [71]:
dictionary = corpora.Dictionary(df['preprocessed_reviews'])
dictionary.filter_extremes(no_below=2, no_above=0.5)

In [72]:
corpus = [dictionary.doc2bow(text) for text in df['preprocessed_reviews']]

In [73]:
num_topics = 3
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=1
)

In [74]:
top_topics = model.top_topics(corpus)

avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print(avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

-6.323201557379149
[([(0.06306122, '드라마'),
   (0.028905312, '연기'),
   (0.023610018, '배우'),
   (0.018458145, '캐릭터'),
   (0.015033362, '시즌'),
   (0.012116957, '작품'),
   (0.011486614, '마지막'),
   (0.011417302, '연출'),
   (0.009633915, '스토리'),
   (0.009543395, '영화'),
   (0.008729128, '생각'),
   (0.00806357, '느낌'),
   (0.007930453, '정도'),
   (0.007891864, '장면'),
   (0.007520974, '작가'),
   (0.0074149324, '대사'),
   (0.007387577, '소재'),
   (0.007080211, '전개'),
   (0.0069193956, '기대'),
   (0.0068660593, '한국')],
  -2.2747032776193),
 ([(0.033957846, '사람'),
   (0.019578183, '사랑'),
   (0.016720966, '생각'),
   (0.012280308, '마음'),
   (0.010967652, '현실'),
   (0.010842953, '인간'),
   (0.009854927, '행복'),
   (0.009655436, '세상'),
   (0.009611256, '이야기'),
   (0.008878749, '아이'),
   (0.00853947, '모습'),
   (0.008478931, '선생'),
   (0.0065661864, '사회'),
   (0.0063739573, '여자'),
   (0.0062463135, '자신'),
   (0.0060978318, '인생'),
   (0.005763534, '순간'),
   (0.005757514, '이유'),
   (0.0054828245, '남자'),
   (0.0053675

### LDA 시각화

In [75]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [76]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(vis)